In [39]:
import os

In [40]:
%pwd

'c:\\DL\\Project_Resume\\Chicken_Disease_Classification'

In [41]:
os.chdir("c:\\DL\\Project_Resume\\Chicken_Disease_Classification")

In [42]:
# entity for callback
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [43]:
from Chicken_Disease_Classification.constants import *
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories

In [44]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [45]:
# component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [46]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        # Ensure the TensorBoard log directory exists
        os.makedirs(tb_running_log_dir, exist_ok=True)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        # Convert Path to string
        checkpoint_filepath = str(self.config.checkpoint_model_filepath)
        
        # Update file path to end with .keras
        if not checkpoint_filepath.endswith('.keras'):
            checkpoint_filepath = checkpoint_filepath.replace('.h5', '.keras')
        
        # Ensure the checkpoint directory exists
        checkpoint_dir = os.path.dirname(checkpoint_filepath)
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [47]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-09-03 06:50:54,210] : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-09-03 06:50:54,214] : INFO : common : yaml file: params.yaml loaded successfully]
[2024-09-03 06:50:54,215] : INFO : common : created directory at: artifacts]
[2024-09-03 06:50:54,217] : INFO : common : created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-03 06:50:54,219] : INFO : common : created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
